In [1]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    val type = getType(name)
    override fun toString(): String = "$name: $score ± $error ns/op"
}

enum class Type { Int, Long, Float, Double }

fun getType(name: String) = when {
    name.contains("Float") -> Type.Float
    name.contains("Int") -> Type.Int
    name.contains("Double") -> Type.Double
    name.contains("Long") -> Type.Long // Long must be the last as it is contained in other test names
    else -> error("Unknown type: ${name}")
}

fun readHotspotBenchmarkData(benchmarkName: String) = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("$benchmarkName\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            name = match.groups[1]!!.value,
            score = match.groups[2]!!.value.toDouble(),
            error = match.groups[3]!!.value.toDouble(),
        )
    }.sortedBy { it.type }
(readHotspotBenchmarkData("Wolf3dBenchmark") + readHotspotBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineFloat: 4.9087861753E7 ± 195844.365 ns/op
inlineFloat: 4.885733804E7 ± 489046.649 ns/op
longPackFloat: 8.311550343E7 ± 140224.653 ns/op
mutableRefFloatIn1Long: 9.4012622697E7 ± 2415715.124 ns/op
mutableRefFloatIn2Longs: 5.6710156491E7 ± 1826058.309 ns/op
valueFloat: 3.6907016518E7 ± 37524.791 ns/op
valueInlineFloat: 3.7185954413E7 ± 160470.782 ns/op
valuePreserveBoxFloat: 3.6009581147E7 ± 1854930.088 ns/op
baselineDouble: 5.0412366376E7 ± 1079493.27 ns/op
inlineDouble: 5.1395603639E7 ± 355671.909 ns/op
mutableRefDouble: 5.8632792455E7 ± 109080.116 ns/op
valueDouble: 3.582699471E7 ± 1160759.908 ns/op
valueInlineDouble: 3.6537176037E7 ± 248733.877 ns/op
valuePreserveBoxDouble: 3.5883592673E7 ± 1037520.526 ns/op

In [2]:
%use lets-plot

In [3]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

fun makePlot(name: String, platform: String, benchmarkLines: List<BenchmarkData>) {
    if (benchmarkLines.isEmpty()) return
    val names = benchmarkLines.map { it.name }
    val scores = benchmarkLines.map { it.score }
    val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
    val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
    val baseline = benchmarkLines.filter { it.name.startsWith("baseline") }.groupBy { it.type }.mapValues { (_, v) -> v.single() }
    val maxScore = scoresPlusErrors.max()
    val plot = letsPlot() +
            labs("$name ($platform)", fill = "Benchmark", y = "ns/op") +
            geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
                x = names
                y = scores
                fill = names
            } +
            geomErrorBar(
                width = 0.5,
                position = PosOptions(kind = PosKind.DODGE),
                size = 0.6
            ) {
                x = names
                ymin = scoresMinusErrors
                ymax = scoresPlusErrors
            } +
            geomText(
                stat = StatOptions(kind = StatKind.IDENTITY),
                position = positionStack(vjust = 1.0),
                size = 4.5,
            ) {
                x = names
                y = scores.map {
                    val threshold = 0.08
                    if (it < maxScore * threshold) it + threshold * maxScore / 2 else it / 2
                }
                label = benchmarkLines.map {
                    val score = it.score / baseline[it.type]!!.score
                    val error = it.error / baseline[it.type]!!.score
                    "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${it.type}"
                }
            } +
            scaleXDiscrete(
                labels = names,
                name = "",
                limits = names + listOf(""),
                breaks = names.toList()
            ) +
            scaleYContinuous(limits = 0 to maxScore, name = "ns/op") +
            theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
            ggsize(1000, 800)
    ggsave(plot = plot, filename = "$name ($platform).svg")
    plot.show()
}


In [4]:
makePlot("Wolf3d", "Hotspot", readHotspotBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "Hotspot", readHotspotBenchmarkData("AckermannBenchmark"))
makePlot("BoxRecreation", "Hotspot", readHotspotBenchmarkData("BoxRecreationBenchmark"))

In [5]:
import java.io.FileFilter
import kotlin.io.path.listDirectoryEntries

fun readArtBenchmarkData(benchmarkName: String) = Path.of("android-benchmark/build/outputs/androidTest-results/connected")
    .toFile().listFiles(FileFilter { it.isDirectory })!!.single()
    .listFiles { dir, name -> name.startsWith("logcat-org.jetbrains.") }!!.flatMap { it.readLines() }
    .mapNotNull {
        val pattern = Regex("^\\d{2}-\\d{2} +\\d+:\\d+:\\d+.\\d+ +\\d+ +\\d+ +I +Benchmark: +$benchmarkName\\.([^\\[]+)\\[Metric \\(timeNs\\) +results: +median +([^,]+), +min [^,]+, +max [^,]+, +standardDeviation: +([^,]+), +.*")
        val match = pattern.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            name = match.groups[1]!!.value,
            score = match.groups[2]!!.value.toDouble(),
            error = match.groups[3]!!.value.toDouble(),
        )
     }.let { lines ->
         val indexes = readHotspotBenchmarkData(benchmarkName).mapIndexed { index, benchmarkData -> benchmarkData.name to index }.toMap()
         if (lines.all { it.name in indexes }) lines.sortedBy { indexes[it.name]!! } else lines.sortedBy { it.type }
     }
(readArtBenchmarkData("Wolf3dBenchmark") + readArtBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineFloat: 4.00157115E8 ± 4735939.996622451 ns/op
inlineFloat: 6.456765765E8 ± 1.0298151356511936E7 ns/op
longPackFloat: 1.17772673E8 ± 909860.8247409129 ns/op
mutableRefFloatIn1Long: 2.21949558E8 ± 277593.34878945735 ns/op
mutableRefFloatIn2Longs: 1.018417885E8 ± 1526716.218396278 ns/op
valueFloat: 2.708624035E8 ± 9105529.563692564 ns/op
valueInlineFloat: 1.76175501E7 ± 92765.14436892745 ns/op
valuePreserveBoxFloat: 2.758951535E8 ± 7728145.790369684 ns/op
baselineDouble: 4.00080519E8 ± 1.1993105733045664E7 ns/op
inlineDouble: 1.0867619605E9 ± 2.062175328498758E7 ns/op
mutableRefDouble: 1.483998845E8 ± 926602.294472071 ns/op
valueDouble: 4.401975575E8 ± 7574697.719579655 ns/op
valueInlineDouble: 2.2745956625E7 ± 30365.671962527293 ns/op
valuePreserveBoxDouble: 4.178391535E8 ± 6403710.467099603 ns/op

In [6]:
makePlot("Wolf3d", "ART", readArtBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "ART", readArtBenchmarkData("AckermannBenchmark"))
makePlot("BoxRecreation", "ART", readArtBenchmarkData("BoxRecreationBenchmark"))